# Decision Tree Classifier
 <a class="anchor" id="1.5"></a>

# Importamos Librerias

In [6]:
#importamos el dataset
import pandas as pd
URL = "../data/processed/BRFSS2021T.csv"
df_diabetes = pd.read_csv(URL)
df_diabetes

,Unnamed: 0,sex,age,race,HTA,fruta_verdura,marital_status,metropolitan_status,height,weight,...,smoker,general_health,coronary_heart_disease,mental_health_past_30_days,phys_health_past_30_days,kidney_disease,depression,blind,cognitive_diff,diagnosed_diabetes
0,0,2,8,-1.0,3.0,101.0,1.0,1.0,1.4986,32.658624,...,3,5.0,2.0,10.0,20.0,2.0,2.0,2.0,2.0,2.0
1,1,2,7,-1.0,1.0,101.0,-1.0,1.0,1.6764,-1.000000,...,4,3.0,1.0,88.0,88.0,1.0,2.0,1.0,1.0,1.0
2,2,2,8,-1.0,1.0,101.0,3.0,1.0,1.6510,77.110640,...,4,2.0,1.0,88.0,88.0,2.0,2.0,2.0,2.0,1.0
3,3,2,6,-1.0,1.0,203.0,1.0,1.0,1.6256,88.450440,...,4,2.0,2.0,10.0,88.0,2.0,2.0,2.0,2.0,1.0
4,4,1,9,-1.0,-1.0,101.0,1.0,2.0,1.8034,93.439952,...,4,5.0,-1.0,88.0,30.0,2.0,2.0,2.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390138,438688,2,0,-1.0,1.0,210.0,5.0,-1.0,1.5748,61.234920,...,3,2.0,2.0,20.0,88.0,2.0,1.0,2.0,2.0,1.0
390139,438689,1,10,-1.0,1.0,101.0,3.0,-1.0,1.5748,-1.000000,...,4,3.0,2.0,88.0,88.0,2.0,2.0,1.0,1.0,1.0
390140,438690,1,4,-1.0,3.0,102.0,1.0,-1.0,1.8034,99.790240,...,4,2.0,2.0,88.0,88.0,2.0,2.0,2.0,2.0,2.0
390141,438691,1,7,-1.0,1.0,101.0,2.0,-1.0,1.8288,79.378600,...,4,2.0,2.0,88.0,88.0,2.0,2.0,2.0,2.0,2.0


# Model

In [7]:
# Iniciamos el modelo
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

In [8]:
#Definimos nuetras X e Y
X = df_diabetes.iloc[:, :-1].values
Y = df_diabetes['diagnosed_diabetes'].values.reshape(-1,1)

In [10]:
#iniciamos el SKF
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, Y)
lst_accu_stratified = []
tree_scores = {'accuracy_scores':[],'precision_scores':[],'recall_scores':[]}

In [5]:
#bucle
for train_index, test_index in skf.split(X, Y):  
    #Separacion del modelo
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    #Entrenamiento
    ModeloDTC=model.fit(X_train, Y_train)
    #Calculo de metricas para cada entrenamiento
    lst_accu_stratified.append(model.score(X_test, Y_test))
    tree_scores['precision_scores'].append(precision_score(y_test,tree_prediction))
    tree_scores['recall_scores'].append(recall_score(y_test,tree_prediction))


KeyboardInterrupt



In [ ]:
#Mostramos resultado
from numpy import mean
print('Overall Accuracy:',mean(lst_accu_stratified))

# Optuna Tunning

In [ ]:
from sklearn.model_selection import cross_val_score
def dtree_objective(trial):
    
    md = trial.suggest_int('max_depth', 2, 64)
    mi = trial.suggest_int('min_inst', 1, 32)
    
    clf =  DecisionTreeClassifier(max_depth=md, min_samples_leaf=mi, random_state=1, )
    scores = cross_val_score(clf, X_train, Y_train, cv=5, scoring='roc_auc')
    
    return scores.mean()
    
dtree_study = optuna.create_study(direction='maximize')
dtree_study.optimize(dtree_objective, n_trials=20)

In [ ]:
#vemos los resultados
print(dtree_study.best_value)
print(dtree_study.best_params)

In [ ]:
#Visualizamos los resultados
from optuna.visualization import plot_optimization_history, plot_slice
plot_optimization_history(dtree_study)

In [ ]:
plot_slice(dtree_study)